[View in Colaboratory](https://colab.research.google.com/github/Curiousss/FLD/blob/master/FLDNetwork.ipynb)

In [1]:
!wget https://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip

--2018-09-01 05:43:04--  https://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip
Resolving ibug.doc.ic.ac.uk (ibug.doc.ic.ac.uk)... 146.169.13.66
Connecting to ibug.doc.ic.ac.uk (ibug.doc.ic.ac.uk)|146.169.13.66|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: http://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip/ [following]
URL transformed to HTTPS due to an HSTS policy
--2018-09-01 05:43:05--  https://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip/
Reusing existing connection to ibug.doc.ic.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘300w_cropped.zip’

300w_cropped.zip        [ <=>                ]   2.82K  --.-KB/s    in 0s      

2018-09-01 05:43:05 (187 MB/s) - ‘300w_cropped.zip’ saved [2885]



In [7]:
!ls
!unzip 300w_cropped.zip
!ls

300w_cropped.zip  sample_data
Archive:  300w_cropped.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of 300w_cropped.zip or
        300w_cropped.zip.zip, and cannot find 300w_cropped.zip.ZIP, period.
300w_cropped.zip  sample_data


In [26]:
from google.colab import files
uploaded = files.upload()

Saving kernel45.txt to kernel45.txt


In [11]:
!ls
!mkdir 300w_samples
!mv indoor* 300w_samples
!ls
!ls 300w_samples

300w_cropped.zip  indoor_003.pts  indoor_006.pts  indoor_009.pts
indoor_001.png	  indoor_004.png  indoor_007.png  indoor_010.png
indoor_001.pts	  indoor_004.pts  indoor_007.pts  indoor_010.pts
indoor_002.png	  indoor_005.png  indoor_008.png  sample_data
indoor_002.pts	  indoor_005.pts  indoor_008.pts
indoor_003.png	  indoor_006.png  indoor_009.png
300w_cropped.zip  indoor_003.png  indoor_006.png  indoor_009.png
300w_samples	  indoor_003.pts  indoor_006.pts  indoor_009.pts
indoor_001.png	  indoor_004.png  indoor_007.png  indoor_010.png
indoor_001.pts	  indoor_004.pts  indoor_007.pts  indoor_010.pts
indoor_002.png	  indoor_005.png  indoor_008.png  sample_data
indoor_002.pts	  indoor_005.pts  indoor_008.pts
indoor_001.png	indoor_003.png	indoor_005.png	indoor_007.png	indoor_009.png
indoor_001.pts	indoor_003.pts	indoor_005.pts	indoor_007.pts	indoor_009.pts
indoor_002.png	indoor_004.png	indoor_006.png	indoor_008.png	indoor_010.png
indoor_002.pts	indoor_004.pts	indoor_006.pts	indoor_008.pts	i

In [19]:
#!git clone https://github.com/albanie/pts_loader
!ls pts_loader/
!cp pts_loader/pts_loader.py .
!ls

LICENSE.md  pts_loader.py  README.md  tests
300w_cropped.zip  300w_samples	pts_loader  pts_loader.py  sample_data


In [13]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
import cv2



Using TensorFlow backend.


In [0]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0
decay = 0
epochs = 100
droupout= 0.1
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1

In [0]:
# Get files
import pts_loader

img_x = []
pts_y = []
for i in range(1,10):
    if i < 10:
        img_f = r"300w_samples/indoor_00" + str(i) +".png"
        pts_f = r"300w_samples/indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"300w_samples/indoor_0" + str(i) +".png"
        pts_f = r"300w_samples/indoor_0" + str(i) +".pts"
    else:
        img_f = r"300w_samples/indoor_" + str(i) +".png"
        pts_f = r"300w_samples/indoor_" + str(i) +".pts"
    pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.imread(img_f)
    
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    img = cv2.resize(img, (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = img.reshape(img_rows, img_cols, 1)
    landmarks = []
    n = 0
    for p in pts:
        new_pts = []
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        #new_points.append(new_pts)
        x = int(new_pts[0])
        y = int(new_pts[1])
        #print(n, x, y)
        landmarks.append(np.zeros((96, 96), np.uint8))
        landmarks[n][x][y] = 255 #WHite
        n = n +1
    img_x.append(img)
    pts_y.append(landmarks)


In [22]:
print(img_x[0].shape)

train_x = []
train_y = []
test_x = []
test_y = []


(96, 96, 1)


In [0]:
train_x = np.array(img_x[:8])
train_y = np.array(pts_y[:8])
test_x = np.array(img_x[8:])
test_y = np.array(pts_y[8:])


In [46]:
print(train_y.shape)
train_y = train_y.reshape(8, 96,96,68)
test_y = test_y.reshape(-1, 96,96,68)

#train_y = K.expand_dims(train_y, 1)
print(train_y.shape)

(8, 96, 96, 68)
(8, 96, 96, 68)


In [0]:
f = open(r'kernel45.txt')
kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")

In [91]:
from keras.layers import Dense, Dropout, Activation, Add
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels, 1))
    #heatmaps = Convolution2D(local_out, kernel_stacked)
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)

    groups = []
    #_d = 1
    for j in range(nLabels):
        #group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
        #groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
        groups.append(tf.nn.conv2d(local_out, kernel_tensor, strides=[1,1,1,1], padding="SAME"))
    
    heatmaps = tf.concat(groups, 0)
    heatmaps = tf.reshape(heatmaps, (-1, 96, 96, 68))
    print("Heatmaps", heatmaps)
    return heatmaps


'''
LOCAL SUBNET
'''

# [ImageC x ImageH x ImageW]

input_img = Input(( img_rows, img_cols, img_channels),)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(input_img)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)

model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
print("Local subnet", model)
'''
1x1 Convolution with activation
'''
model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation='relu')(model)
model_Conv = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels))(model_Conv)
print("Main 1x1", model_Conv)
'''
Ol 1x1 Linear convolution
Apply Gaussian Kernel filter
'''
model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None)(model)
# What about tiling the kernels????
model_Ol = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels), name='Ol')(model_Ol)
print("Local output", model_Ol)

output_Ol = keras.layers.concatenate([model_Conv, model_Ol])
print("Concatenate", output_Ol)
output_Ol = Reshape((96, 96, -1))(output_Ol)
#print("Reshape", output_Ol)

'''
GLOBAL SUBNET
'''
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(output_Ol)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nLabels, kernel_size=(1, 1), activation=None)(model)
FLD_model = Model(inputs = input_img, outputs=model)
FLD_model.summary()


Local subnet Tensor("batch_normalization_460/cond/Merge:0", shape=(?, 96, 96, 128), dtype=float32)
local_out Tensor("conv2d_522/Relu:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("lambda_26/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("lambda_26/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Main 1x1 Tensor("lambda_26/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
local_out Tensor("conv2d_523/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("Ol_18/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("Ol_18/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Local output Tensor("Ol_18/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Concatenate Tensor("concatenate_19/concat:0", shape=(?, 96, 96, 136), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLa

* Loss Function: Weighted Square Error
* Exclude non-labeled landmarks from the loss
* Weight each of N landmarks depending on whether they lie within the image boundary:
\begin{equation*}
wij = V (i,j) + \frac{L(i,j)}{10} + \frac{L(i,j)}{2N}\sum_{k=1}^N V (i,k),       \\[1pt]
\text{    where V (i; j) and L(i; j) are indicator functions with L(i; j) = 1 iff landmark j in face i is labeled}    \\[1pt]
\text {V (i; j) = 1 iff landmark j in face i is within the image boundary, zero otherwise.    }\\[1pt]
\text{Note that L(i; j) = 0 implies V (i; j) = 0.}\end{equation*}

\begin{equation*}
e_i = \sum_{n=1}^N w_{in}(((O^l_n - G_n) * K)^2 + (O^g_n - G_n * K)^2), \\[2pt]
\text{    where    } \\[2pt]
O^l_n    and    O^g_n = \text{nth channels of the local- and global context subnet outputs, respectively.} \\[1pt]
\text{K = kernel }   \\[1pt]
\text{Gn = ground truth heatmap for landmark n }   \\[1pt]
\text{For efficiency, the kernel convolution after stacking is reused for the loss}
\end{equation*}


In [0]:
import tensorflow as tf
from keras import backend as K

def landmark_weight(labels):
    #[numLabels x 1 x 1]
    Win = tf.reduce_mean(labels)
    Win = tf.multiply(Win, 0.5)
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without heatmaps
'''
def loss_function(labels, Og):#Ol, Og,labels):
    
    # Gn = labels x K
    #Gn = apply_kernel(labels)
    #Gn = labels

    #Win = landmark_weight(labels)
    print("---------------Loss Function----------------")
    print("Og", Og)
    print("Labels", labels)
    # Og - Gn
    global_diff = tf.subtract(Og, labels)
    global_err = tf.square(global_diff)
    
    # Ol - Gn
    '''
    Ol = FLD_model.get_layer(name='Ol')...
    How to access intermediate layer or model from here???
    Currently accessing since model from global...
    
    local_diff = tf.subtract(model_Ol, Gn)
    local_err = tf.square(local_diff)
    
    comb_err = tf.add(local_err, global_err)
    
    wt_square_err = tf.reduce_sum(Win * comb_err)
    return wt_square_err    
    '''
    return tf.square(global_err)
    

In [96]:
FLD_model.compile(optimizer='sgd',
              loss=loss_function,
              metrics=['accuracy'])


---------------Loss Function----------------
Og Tensor("conv2d_531/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
Labels Tensor("conv2d_531_target_1:0", shape=(?, ?, ?, ?), dtype=float32)


In [97]:
 FLD_model.fit(train_x, train_y, batch_size=8, epochs=1, validation_data=(test_x, test_y))

Train on 8 samples, validate on 1 samples
Epoch 1/1
8/8 [==============================] - 15s 2s/step - loss: 520200.6562 - acc: 0.0190 - val_loss: inf - val_acc: 1.0851e-04


In [0]:
import cntk as C

cntk_model = C.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\python\output\models\model.21")

In [0]:
cntk_model.parameters

In [0]:
cntk_model.save("cntk_model.h5")

In [0]:
FLD = keras.models.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\cntk_model.h5")

In [0]:
import keras.backend as K

#save_model_hdf5(cntk_model, 'my_model.h5')
kerasmodel = K.functional_ops. load_model(cntk_model)

In [0]:
#func = cntk_model.outputs[0]
cntk_model.save('cntk_model.onnx', format=C.ModelFormat.)

In [0]:
import onnx
onnx_model = onnx.load('cntk_model.onnx')

In [0]:
from onnx_tf.backend import prepare
tf_model = prepare(onnx_model)